# Day16_1: MLP (다층 퍼셉트론)

## 학습 목표

**Part 1: 기초**
1. 퍼셉트론의 구조와 동작 원리 이해하기
2. 활성화 함수(Sigmoid, Tanh, ReLU) 비교하기
3. 손실 함수(MSE, Cross-Entropy) 이해하기
4. 경사 하강법 개념 이해하기
5. XOR 문제와 MLP의 필요성 이해하기

**Part 2: 심화**
1. 역전파 알고리즘 개념적으로 이해하기
2. PyTorch로 MLP 구현하기
3. 옵티마이저(SGD, Adam) 비교하기
4. MNIST 손글씨 분류 실습하기

---

## 왜 이것을 배우나요?

| 개념 | 실무 활용 | 예시 |
|------|----------|------|
| 퍼셉트론 | 신경망의 기본 단위 | 모든 딥러닝 모델의 구성 요소 |
| 활성화 함수 | 비선형성 추가 | ReLU로 깊은 네트워크 학습 가능 |
| 손실 함수 | 학습 목표 정의 | 분류/회귀에 맞는 손실 선택 |
| MLP | 정형 데이터 분류/회귀 | 고객 이탈 예측, 가격 예측 |

**분석가 관점**: MLP는 딥러닝의 기본 구조입니다. CNN, RNN, Transformer 모두 MLP를 기반으로 확장됩니다. 이 개념을 확실히 이해하면 모든 딥러닝 아키텍처의 동작 원리를 파악할 수 있습니다!

---

# Part 1: 기초

---

## 1.1 퍼셉트론 (Perceptron)

### 생물학적 뉴런 비유

```
생물학적 뉴런:           인공 뉴런 (퍼셉트론):
  수상돌기 (입력)    ->    입력 (x1, x2, ...)
  세포체 (처리)      ->    가중합 + 편향
  축삭돌기 (출력)    ->    활성화 함수 -> 출력
```

### 퍼셉트론 수식

```
y = f(w1*x1 + w2*x2 + ... + wn*xn + b)
y = f(sum(wi*xi) + b)
y = f(W @ X + b)

- X: 입력 벡터
- W: 가중치 벡터 (학습되는 파라미터)
- b: 편향 (학습되는 파라미터)
- f: 활성화 함수
```

In [ ]:
import torch
import torch.nn as nn
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd
import torch.optim as optim

print(f"PyTorch 버전: {torch.__version__}")

In [ ]:
# 단일 퍼셉트론 구현 (NumPy)
def perceptron(x, w, b, activation='step'):
    """
    단일 퍼셉트론
    x: 입력 벡터
    w: 가중치 벡터
    b: 편향
    activation: 활성화 함수 종류
    """
    # 가중합 계산
    z = np.dot(w, x) + b
    
    # 활성화 함수 적용
    if activation == 'step':  # 계단 함수 (원래 퍼셉트론)
        return 1 if z >= 0 else 0
    elif activation == 'sigmoid':
        return 1 / (1 + np.exp(-z))
    else:
        return z  # 선형 (활성화 없음)

# AND 게이트 구현
print("AND 게이트 (퍼셉트론)")
print("="*30)

# 가중치와 편향 (수동 설정)
w_and = np.array([0.5, 0.5])
b_and = -0.7

# 테스트
inputs = [[0, 0], [0, 1], [1, 0], [1, 1]]
for x in inputs:
    output = perceptron(np.array(x), w_and, b_and, 'step')
    print(f"AND({x[0]}, {x[1]}) = {output}")

In [ ]:
# OR 게이트 구현
print("OR 게이트 (퍼셉트론)")
print("="*30)

w_or = np.array([0.5, 0.5])
b_or = -0.2

for x in inputs:
    output = perceptron(np.array(x), w_or, b_or, 'step')
    print(f"OR({x[0]}, {x[1]}) = {output}")

### 실무 예시: 퍼셉트론의 결정 경계

퍼셉트론은 **선형 결정 경계**를 학습합니다:
- w1*x1 + w2*x2 + b = 0
- 이 직선이 두 클래스를 분리

In [ ]:
# AND 게이트의 결정 경계 시각화
x1_range = np.linspace(-0.5, 1.5, 100)

# 결정 경계: w1*x1 + w2*x2 + b = 0
# x2 = -(w1*x1 + b) / w2
x2_boundary_and = -(w_and[0] * x1_range + b_and) / w_and[1]
x2_boundary_or = -(w_or[0] * x1_range + b_or) / w_or[1]

# 데이터 포인트
points = pd.DataFrame({
    'x1': [0, 0, 1, 1],
    'x2': [0, 1, 0, 1],
    'AND': [0, 0, 0, 1],
    'OR': [0, 1, 1, 1]
})

# 시각화
fig = make_subplots(rows=1, cols=2, subplot_titles=['AND 게이트', 'OR 게이트'])

# AND 게이트
colors_and = ['blue' if y == 0 else 'red' for y in points['AND']]
fig.add_trace(
    go.Scatter(x=points['x1'], y=points['x2'], mode='markers', 
               marker=dict(size=15, color=colors_and), name='AND 데이터'),
    row=1, col=1
)
fig.add_trace(
    go.Scatter(x=x1_range, y=x2_boundary_and, mode='lines',
               line=dict(color='green', dash='dash'), name='결정 경계'),
    row=1, col=1
)

# OR 게이트
colors_or = ['blue' if y == 0 else 'red' for y in points['OR']]
fig.add_trace(
    go.Scatter(x=points['x1'], y=points['x2'], mode='markers',
               marker=dict(size=15, color=colors_or), name='OR 데이터'),
    row=1, col=2
)
fig.add_trace(
    go.Scatter(x=x1_range, y=x2_boundary_or, mode='lines',
               line=dict(color='green', dash='dash'), name='결정 경계'),
    row=1, col=2
)

fig.update_xaxes(range=[-0.5, 1.5], title_text='x1')
fig.update_yaxes(range=[-0.5, 1.5], title_text='x2')
fig.update_layout(title='퍼셉트론의 선형 결정 경계', height=400, showlegend=False)
fig.show()

---

## 실습 퀴즈 정답

**난이도**: (쉬움) ~ (어려움)

---

### Q1. 퍼셉트론 출력 계산하기 

**문제**: 아래 퍼셉트론의 출력을 계산하세요 (Sigmoid 활성화 사용).

- 입력: x = [1.0, 2.0]
- 가중치: w = [0.5, -0.5]
- 편향: b = 0.1

**기대 결과**: Sigmoid 출력 값

In [ ]:
# Q1 정답
import numpy as np

x = np.array([1.0, 2.0])
w = np.array([0.5, -0.5])
b = 0.1

# 1. 가중합 계산: z = w @ x + b
z = np.dot(w, x) + b
print(f"가중합 z = w @ x + b")
print(f"z = {w[0]}*{x[0]} + {w[1]}*{x[1]} + {b}")
print(f"z = {w[0]*x[0]} + {w[1]*x[1]} + {b}")
print(f"z = {z}")

# 2. Sigmoid 활성화 적용
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

output = sigmoid(z)
print(f"\nSigmoid(z) = 1 / (1 + exp(-{z}))")
print(f"출력: {output:.4f}")

In [ ]:
# 테스트 케이스로 검증
expected_z = 0.5*1.0 + (-0.5)*2.0 + 0.1  # = 0.5 - 1.0 + 0.1 = -0.4
expected_output = 1 / (1 + np.exp(0.4))  # sigmoid(-0.4)

assert abs(z - (-0.4)) < 1e-6, f"가중합 계산 오류: {z}"
assert abs(output - expected_output) < 1e-6, f"Sigmoid 계산 오류: {output}"
print("모든 테스트 통과!")

### 풀이 설명

**접근 방법**: 퍼셉트론의 계산 과정을 순서대로 수행합니다.

**핵심 개념**:
1. **가중합**: z = w1*x1 + w2*x2 + b = 0.5*1.0 + (-0.5)*2.0 + 0.1 = -0.4
2. **Sigmoid**: f(z) = 1 / (1 + exp(-z)) = 1 / (1 + exp(0.4)) = 0.4013

**대안 솔루션**:
```python
# PyTorch로 계산
import torch
x_t = torch.tensor([1.0, 2.0])
w_t = torch.tensor([0.5, -0.5])
b_t = torch.tensor(0.1)
output_t = torch.sigmoid(torch.dot(w_t, x_t) + b_t)
```

**흔한 실수**: 가중합 계산 시 순서 혼동 (w @ x가 아닌 x @ w로 계산)

**실무 팁**: Sigmoid 출력이 0.5보다 크면 클래스 1, 작으면 클래스 0으로 분류합니다.

---

### Q2. 활성화 함수 선택하기 

**문제**: 다음 상황에 적합한 활성화 함수를 선택하고, 이유를 설명하세요.

1. 은닉층 (일반적인 경우)
2. 이진 분류의 출력층 (0~1 확률)
3. 다중 분류의 출력층 (여러 클래스 중 하나 선택)

In [ ]:
# Q2 정답
print("활성화 함수 선택 가이드")
print("="*60)

answers = {
    "1. 은닉층 (일반적인 경우)": {
        "활성화 함수": "ReLU (Rectified Linear Unit)",
        "이유": [
            "계산이 빠름 (max(0, x))",
            "기울기 소실 문제 완화 (양수 영역에서 기울기 = 1)",
            "희소 활성화로 효율적 학습",
            "딥러닝의 사실상 표준"
        ],
        "PyTorch": "nn.ReLU()"
    },
    "2. 이진 분류 출력층": {
        "활성화 함수": "Sigmoid",
        "이유": [
            "출력 범위가 (0, 1)로 확률 해석 가능",
            "이진 분류에서 클래스 1의 확률 P(y=1|x)",
            "임계값 0.5로 분류 결정"
        ],
        "PyTorch": "nn.Sigmoid() + nn.BCELoss()"
    },
    "3. 다중 분류 출력층": {
        "활성화 함수": "Softmax",
        "이유": [
            "모든 출력의 합이 1 (확률 분포)",
            "각 클래스의 확률 P(y=k|x)",
            "가장 높은 확률의 클래스 선택"
        ],
        "PyTorch": "nn.CrossEntropyLoss() (Softmax 포함)"
    }
}

for situation, info in answers.items():
    print(f"\n{situation}")
    print(f"  정답: {info['활성화 함수']}")
    print(f"  이유:")
    for reason in info['이유']:
        print(f"    - {reason}")
    print(f"  PyTorch: {info['PyTorch']}")

In [ ]:
# 실제 사용 예시
import torch.nn as nn

# 이진 분류 모델
binary_classifier = nn.Sequential(
    nn.Linear(10, 32),
    nn.ReLU(),          # 은닉층: ReLU
    nn.Linear(32, 1),
    nn.Sigmoid()        # 출력층: Sigmoid
)

# 다중 분류 모델
multi_classifier = nn.Sequential(
    nn.Linear(10, 32),
    nn.ReLU(),          # 은닉층: ReLU
    nn.Linear(32, 5)    # 출력층: Softmax는 CrossEntropyLoss에 포함
)

print("이진 분류 모델:")
print(binary_classifier)
print("\n다중 분류 모델:")
print(multi_classifier)

### 풀이 설명

**접근 방법**: 각 상황의 요구사항에 맞는 활성화 함수의 특성을 매칭합니다.

**핵심 개념**:
- **ReLU**: max(0, x), 은닉층 기본
- **Sigmoid**: 1/(1+e^-x), 0~1 확률
- **Softmax**: 다중 클래스 확률 분포

**대안 솔루션**:
```python
# 은닉층 대안
nn.LeakyReLU(0.1)  # dying ReLU 방지
nn.GELU()          # Transformer에서 사용
nn.Tanh()          # RNN 은닉층
```

**흔한 실수**: CrossEntropyLoss는 내부에 Softmax가 포함되어 있어, 모델 출력에 Softmax를 추가하면 중복 적용됩니다.

**실무 팁**: ReLU 변형 중 GELU는 최근 Transformer 기반 모델에서 많이 사용됩니다.

---

### Q3. 손실 함수 계산하기 

**문제**: 다음 예측과 실제 값으로 BCE Loss를 수동 계산하세요.

- 예측 확률: y_pred = [0.9, 0.3, 0.8]
- 실제 레이블: y_true = [1, 0, 1]

In [ ]:
# Q3 정답
import numpy as np

y_pred = np.array([0.9, 0.3, 0.8])
y_true = np.array([1, 0, 1])

# BCE Loss 수식: -[y*log(p) + (1-y)*log(1-p)]
def bce_loss_manual(y_pred, y_true):
    epsilon = 1e-15  # log(0) 방지
    y_pred = np.clip(y_pred, epsilon, 1 - epsilon)
    
    losses = []
    for p, y in zip(y_pred, y_true):
        loss = -(y * np.log(p) + (1 - y) * np.log(1 - p))
        losses.append(loss)
        print(f"  y={y}, p={p:.1f}: -{y}*log({p:.1f}) - {1-y}*log({1-p:.1f}) = {loss:.4f}")
    
    return np.mean(losses)

print("BCE Loss 계산 과정:")
bce = bce_loss_manual(y_pred, y_true)
print(f"\n평균 BCE Loss: {bce:.4f}")

In [ ]:
# PyTorch로 검증
import torch
import torch.nn as nn

y_pred_t = torch.tensor([0.9, 0.3, 0.8])
y_true_t = torch.tensor([1.0, 0.0, 1.0])

bce_loss = nn.BCELoss()
pytorch_bce = bce_loss(y_pred_t, y_true_t)

print(f"수동 계산: {bce:.4f}")
print(f"PyTorch 계산: {pytorch_bce.item():.4f}")

# 테스트
assert abs(bce - pytorch_bce.item()) < 1e-4, "BCE 계산 오류"
print("\n모든 테스트 통과!")

### 풀이 설명

**접근 방법**: BCE Loss 공식을 각 샘플에 적용하고 평균을 구합니다.

**핵심 개념**:
- BCE Loss = -[y*log(p) + (1-y)*log(1-p)]
- y=1일 때: -log(p), 확률이 높을수록 손실 작음
- y=0일 때: -log(1-p), 확률이 낮을수록 손실 작음

**대안 솔루션**:
```python
# 벡터화 계산
bce = -np.mean(y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred))
```

**흔한 실수**: log(0)은 -infinity이므로 epsilon으로 클리핑해야 합니다.

**실무 팁**: PyTorch의 BCEWithLogitsLoss는 Sigmoid + BCE를 합쳐서 수치적으로 더 안정적입니다.

---

### Q4. 경사 하강법 구현하기 

**문제**: 함수 f(x) = (x - 3)^2의 최솟값을 경사 하강법으로 찾으세요.

- 시작점: x = 10
- 학습률: 0.1
- 반복 횟수: 50

In [ ]:
# Q4 정답
import numpy as np

# 함수 정의
def f(x):
    return (x - 3) ** 2

# 미분: df/dx = 2(x - 3)
def gradient(x):
    return 2 * (x - 3)

# 경사 하강법
x = 10.0  # 시작점
lr = 0.1  # 학습률
iterations = 50

x_history = [x]
loss_history = [f(x)]

print("경사 하강법 실행")
print("="*50)
print(f"시작점: x = {x}, f(x) = {f(x)}")
print(f"목표: x = 3 (최솟값)")

for i in range(iterations):
    grad = gradient(x)
    x = x - lr * grad  # 경사 하강
    
    x_history.append(x)
    loss_history.append(f(x))
    
    if i < 5 or i == iterations - 1:
        print(f"Iteration {i+1:2d}: x = {x:.6f}, f(x) = {f(x):.6f}, grad = {grad:.4f}")
    elif i == 5:
        print("  ...")

print(f"\n최종 결과: x = {x:.6f} (목표: 3.0)")
print(f"최솟값: f(x) = {f(x):.10f} (목표: 0.0)")

In [ ]:
# 시각화
import plotly.graph_objects as go

x_range = np.linspace(-2, 12, 100)
y_range = f(x_range)

fig = go.Figure()

# 함수 곡선
fig.add_trace(go.Scatter(x=x_range, y=y_range, mode='lines', name='f(x) = (x-3)^2'))

# 경사 하강 경로
fig.add_trace(go.Scatter(
    x=x_history, y=loss_history, mode='markers+lines',
    marker=dict(size=8, color='red'),
    name='경사 하강 경로'
))

# 최솟값 표시
fig.add_trace(go.Scatter(
    x=[3], y=[0], mode='markers',
    marker=dict(size=15, color='green', symbol='star'),
    name='최솟값 (x=3)'
))

fig.update_layout(
    title='경사 하강법: f(x) = (x-3)^2의 최솟값 찾기',
    xaxis_title='x',
    yaxis_title='f(x)',
    template='plotly_white'
)
fig.show()

In [ ]:
# 테스트
assert abs(x - 3.0) < 0.01, f"x가 3에 수렴하지 않음: {x}"
assert abs(f(x) - 0.0) < 0.0001, f"최솟값에 도달하지 않음: {f(x)}"
print("모든 테스트 통과!")

### 풀이 설명

**접근 방법**: 경사 하강법 공식 x_new = x_old - lr * gradient를 반복 적용합니다.

**핵심 개념**:
- 함수: f(x) = (x - 3)^2
- 미분: df/dx = 2(x - 3)
- 최솟값: x = 3에서 f(x) = 0
- 업데이트: x = x - 0.1 * 2(x - 3) = x - 0.2(x - 3) = 0.8x + 0.6

**대안 솔루션**:
```python
# PyTorch Autograd 사용
x = torch.tensor([10.0], requires_grad=True)
for _ in range(50):
    loss = (x - 3) ** 2
    loss.backward()
    with torch.no_grad():
        x -= 0.1 * x.grad
        x.grad.zero_()
```

**흔한 실수**: 학습률이 너무 크면 발산, 너무 작으면 수렴이 느립니다.

**실무 팁**: 실제 딥러닝에서는 학습률 스케줄러를 사용하여 학습 중 학습률을 조절합니다.

---

### Q5. XOR MLP 수정하기 

**문제**: XOR 문제를 해결하는 MLP를 은닉층 2개로 수정하세요.

- 구조: 입력(2) -> 은닉(8) -> 은닉(4) -> 출력(1)
- 활성화: ReLU (은닉), Sigmoid (출력)

In [ ]:
# Q5 정답
import torch
import torch.nn as nn
import torch.optim as optim

# XOR 데이터
X_xor = torch.tensor([[0, 0], [0, 1], [1, 0], [1, 1]], dtype=torch.float32)
y_xor = torch.tensor([[0], [1], [1], [0]], dtype=torch.float32)

# 2-은닉층 MLP
class XOR_MLP_v2(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(2, 8),    # 입력(2) -> 은닉1(8)
            nn.ReLU(),
            nn.Linear(8, 4),    # 은닉1(8) -> 은닉2(4)
            nn.ReLU(),
            nn.Linear(4, 1),    # 은닉2(4) -> 출력(1)
            nn.Sigmoid()
        )
    
    def forward(self, x):
        return self.model(x)

# 모델 생성
model_v2 = XOR_MLP_v2()
print("XOR MLP v2 (2-은닉층):")
print(model_v2)
print(f"\n총 파라미터 수: {sum(p.numel() for p in model_v2.parameters())}")

In [ ]:
# 학습
criterion = nn.BCELoss()
optimizer = optim.Adam(model_v2.parameters(), lr=0.1)

epochs = 1000
for epoch in range(epochs):
    y_pred = model_v2(X_xor)
    loss = criterion(y_pred, y_xor)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if epoch % 200 == 0:
        print(f"Epoch {epoch:4d}: Loss = {loss.item():.4f}")

# 최종 예측
print("\n===== XOR 학습 완료 =====")
with torch.no_grad():
    predictions = model_v2(X_xor)
    for i in range(4):
        x = X_xor[i].numpy()
        pred = predictions[i].item()
        actual = y_xor[i].item()
        pred_class = 1 if pred > 0.5 else 0
        correct = "O" if pred_class == int(actual) else "X"
        print(f"XOR({int(x[0])}, {int(x[1])}) = {pred:.3f} -> {pred_class} (실제: {int(actual)}) [{correct}]")

In [ ]:
# 테스트
with torch.no_grad():
    preds = model_v2(X_xor)
    pred_classes = (preds > 0.5).float()
    accuracy = (pred_classes == y_xor).float().mean()
    
assert accuracy == 1.0, f"XOR 정확도 100%가 아님: {accuracy*100:.0f}%"
print(f"\nXOR 정확도: {accuracy*100:.0f}%")
print("모든 테스트 통과!")

### 풀이 설명

**접근 방법**: nn.Sequential로 2개의 은닉층을 가진 MLP를 구성합니다.

**핵심 개념**:
- 구조: 2 -> 8 -> 4 -> 1
- 은닉층: Linear + ReLU
- 출력층: Linear + Sigmoid (이진 분류)

**대안 솔루션**:
```python
# 클래스 버전
class XOR_MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(2, 8)
        self.fc2 = nn.Linear(8, 4)
        self.fc3 = nn.Linear(4, 1)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        return self.sigmoid(self.fc3(x))
```

**흔한 실수**: XOR 문제는 작은 데이터셋이라 빠르게 과적합됩니다. 실제 문제에서는 검증 데이터로 확인하세요.

**실무 팁**: XOR 문제는 신경망이 비선형 패턴을 학습할 수 있는지 테스트하는 좋은 sanity check입니다.

---

### Q6. 역전파 그래디언트 확인하기 

**문제**: y = 2x^3 - 3x^2 + x 함수에서 x=2일 때 dy/dx를 Autograd로 계산하고, 수동 계산과 비교하세요.

힌트: dy/dx = 6x^2 - 6x + 1

In [ ]:
# Q6 정답
import torch

# x=2에서 그래디언트 계산
x = torch.tensor([2.0], requires_grad=True)

# 함수: y = 2x^3 - 3x^2 + x
y = 2 * x**3 - 3 * x**2 + x

# 역전파
y.backward()

# 수동 계산: dy/dx = 6x^2 - 6x + 1
# x=2: 6*4 - 6*2 + 1 = 24 - 12 + 1 = 13
manual_grad = 6 * (2.0)**2 - 6 * (2.0) + 1

print("함수: y = 2x^3 - 3x^2 + x")
print("미분: dy/dx = 6x^2 - 6x + 1")
print(f"\nx = {x.item()}")
print(f"y = 2*{x.item()}^3 - 3*{x.item()}^2 + {x.item()} = {y.item()}")
print(f"\nAutograd dy/dx: {x.grad.item()}")
print(f"수동 계산 dy/dx: 6*{x.item()}^2 - 6*{x.item()} + 1 = {manual_grad}")

In [ ]:
# 테스트
assert abs(x.grad.item() - 13.0) < 1e-6, f"그래디언트 오류: {x.grad.item()}"
assert abs(x.grad.item() - manual_grad) < 1e-6, "Autograd와 수동 계산 불일치"
print("\n모든 테스트 통과!")

### 풀이 설명

**접근 방법**: PyTorch의 Autograd로 자동 미분하고, 수동 계산과 비교합니다.

**핵심 개념**:
- 원 함수: y = 2x^3 - 3x^2 + x
- 미분: dy/dx = 6x^2 - 6x + 1 (멱함수 미분 규칙)
- x=2 대입: 6(4) - 6(2) + 1 = 24 - 12 + 1 = 13

**대안 솔루션**:
```python
# torch.autograd.grad 직접 사용
grad = torch.autograd.grad(y, x)
print(grad[0].item())
```

**흔한 실수**: backward()는 스칼라에 대해서만 호출 가능합니다. 벡터 출력이면 gradient 인자가 필요합니다.

**실무 팁**: Autograd 결과를 수동 계산과 비교하는 것은 커스텀 레이어 구현 시 디버깅에 유용합니다.

---

### Q7. 옵티마이저 선택하기 

**문제**: 아래 상황에 적합한 옵티마이저를 선택하고 이유를 설명하세요.

1. 빠른 프로토타이핑이 필요한 경우
2. 일반화 성능이 중요한 최종 모델
3. 학습률 튜닝 시간이 부족한 경우

In [ ]:
# Q7 정답
print("옵티마이저 선택 가이드")
print("="*70)

optimizer_guide = {
    "1. 빠른 프로토타이핑": {
        "추천": "Adam",
        "이유": [
            "빠른 수렴 속도",
            "기본 lr=0.001로 대부분 잘 동작",
            "하이퍼파라미터 튜닝 최소화",
            "연구 단계에서 빠른 실험 가능"
        ],
        "PyTorch": "optim.Adam(model.parameters(), lr=0.001)"
    },
    "2. 일반화 성능 중요": {
        "추천": "SGD + Momentum + Weight Decay",
        "이유": [
            "Adam보다 일반화 성능이 좋은 경우가 많음",
            "ImageNet 등 대규모 벤치마크에서 검증됨",
            "학습률 스케줄링과 함께 사용",
            "최종 프로덕션 모델에 권장"
        ],
        "PyTorch": "optim.SGD(model.parameters(), lr=0.1, momentum=0.9, weight_decay=1e-4)"
    },
    "3. 학습률 튜닝 시간 부족": {
        "추천": "Adam 또는 AdamW",
        "이유": [
            "적응적 학습률로 lr에 덜 민감",
            "기본값으로도 안정적 학습",
            "AdamW는 weight decay 개선 버전",
            "Transformer 계열 모델에서 표준"
        ],
        "PyTorch": "optim.AdamW(model.parameters(), lr=0.001, weight_decay=0.01)"
    }
}

for situation, info in optimizer_guide.items():
    print(f"\n{situation}")
    print(f"  추천: {info['추천']}")
    print(f"  이유:")
    for reason in info['이유']:
        print(f"    - {reason}")
    print(f"  PyTorch: {info['PyTorch']}")

In [ ]:
# 실제 사용 예시
import torch.nn as nn
import torch.optim as optim

# 예시 모델
model = nn.Linear(10, 2)

# 상황별 옵티마이저 설정
opt_prototype = optim.Adam(model.parameters(), lr=0.001)
opt_production = optim.SGD(model.parameters(), lr=0.1, momentum=0.9, weight_decay=1e-4)
opt_quick = optim.AdamW(model.parameters(), lr=0.001, weight_decay=0.01)

print("옵티마이저 설정 완료:")
print(f"  프로토타입: {type(opt_prototype).__name__}")
print(f"  프로덕션: {type(opt_production).__name__}")
print(f"  빠른 개발: {type(opt_quick).__name__}")

### 풀이 설명

**접근 방법**: 각 상황의 우선순위(속도/일반화/편의성)에 맞는 옵티마이저를 선택합니다.

**핵심 개념**:
- **Adam**: 빠른 수렴, 적응적 학습률
- **SGD+Momentum**: 느리지만 일반화 좋음
- **AdamW**: Adam + 개선된 weight decay

**대안 솔루션**:
```python
# 학습률 스케줄러와 함께 사용
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=100)

# 또는 OneCycleLR (빠른 학습)
scheduler = optim.lr_scheduler.OneCycleLR(optimizer, max_lr=0.1, epochs=100, steps_per_epoch=len(train_loader))
```

**흔한 실수**: Adam을 사용할 때 weight_decay를 L2 정규화처럼 적용하면 문제가 있음. AdamW 권장.

**실무 팁**: 최근 트렌드는 AdamW + Cosine Annealing 또는 WarmupCosine 조합입니다.

---

### Q8. MLP 구조 설계하기 

**문제**: Fashion MNIST (28x28 이미지, 10개 클래스) 분류를 위한 MLP를 설계하세요.

요구사항:
- 은닉층 3개 (256, 128, 64)
- 드롭아웃 0.3
- BatchNorm 적용

In [ ]:
# Q8 정답
import torch
import torch.nn as nn

class FashionMNIST_MLP(nn.Module):
    def __init__(self, dropout=0.3):
        super().__init__()
        
        self.flatten = nn.Flatten()
        
        self.model = nn.Sequential(
            # 입력층 -> 은닉층1
            nn.Linear(784, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(dropout),
            
            # 은닉층1 -> 은닉층2
            nn.Linear(256, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(dropout),
            
            # 은닉층2 -> 은닉층3
            nn.Linear(128, 64),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Dropout(dropout),
            
            # 은닉층3 -> 출력층
            nn.Linear(64, 10)
        )
    
    def forward(self, x):
        x = self.flatten(x)  # (batch, 1, 28, 28) -> (batch, 784)
        return self.model(x)

# 모델 생성
model = FashionMNIST_MLP(dropout=0.3)
print("Fashion MNIST MLP (3-은닉층 + BatchNorm + Dropout):")
print(model)

In [ ]:
# 파라미터 수 확인
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"\n총 파라미터 수: {count_parameters(model):,}")

# 레이어별 파라미터
print("\n레이어별 파라미터:")
for name, param in model.named_parameters():
    print(f"  {name}: {param.numel():,}")

In [ ]:
# 테스트 (순전파)
x_test = torch.randn(4, 1, 28, 28)  # 배치 4개
model.eval()  # 평가 모드 (BatchNorm, Dropout 비활성화)

with torch.no_grad():
    output = model(x_test)

print(f"\n입력 shape: {x_test.shape}")
print(f"출력 shape: {output.shape}")
assert output.shape == torch.Size([4, 10]), "출력 shape 오류"
print("\n모든 테스트 통과!")

### 풀이 설명

**접근 방법**: 각 은닉층에 Linear -> BatchNorm -> ReLU -> Dropout 순서로 구성합니다.

**핵심 개념**:
- **BatchNorm**: 학습 안정화, 빠른 수렴
- **Dropout**: 과적합 방지
- **순서**: Linear -> BatchNorm -> ReLU -> Dropout (권장)

**대안 솔루션**:
```python
# 레이어 함수로 생성
def make_layer(in_dim, out_dim, dropout):
    return nn.Sequential(
        nn.Linear(in_dim, out_dim),
        nn.BatchNorm1d(out_dim),
        nn.ReLU(),
        nn.Dropout(dropout)
    )
```

**흔한 실수**: BatchNorm은 평가 시 다르게 동작하므로 model.eval() 필수입니다.

**실무 팁**: BatchNorm과 Dropout을 함께 사용할 때 순서가 중요합니다. BatchNorm -> Dropout 순서가 일반적입니다.

---

### Q9. MNIST 정확도 향상하기 

**문제**: MNIST MLP 모델을 수정하여 테스트 정확도 98% 이상을 달성하세요.

힌트:
- 은닉층 크기 증가
- 에포크 증가
- 학습률 스케줄러 사용
- 배치 정규화 추가

In [ ]:
# Q9 정답
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

# 데이터 로드
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

train_dataset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

# 개선된 MLP
class ImprovedMNIST_MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.model = nn.Sequential(
            # 더 큰 은닉층
            nn.Linear(784, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(0.2),
            
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.2),
            
            nn.Linear(256, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(0.2),
            
            nn.Linear(128, 10)
        )
    
    def forward(self, x):
        x = self.flatten(x)
        return self.model(x)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = ImprovedMNIST_MLP().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)

print(f"Device: {device}")
print(f"파라미터 수: {sum(p.numel() for p in model.parameters()):,}")

In [ ]:
# 학습
def train_epoch(model, loader, criterion, optimizer, device):
    model.train()
    total_loss, correct, total = 0, 0, 0
    
    for X_batch, y_batch in loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        _, predicted = outputs.max(1)
        total += y_batch.size(0)
        correct += predicted.eq(y_batch).sum().item()
    
    return total_loss / len(loader), 100. * correct / total

def evaluate(model, loader, criterion, device):
    model.eval()
    total_loss, correct, total = 0, 0, 0
    
    with torch.no_grad():
        for X_batch, y_batch in loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            
            total_loss += loss.item()
            _, predicted = outputs.max(1)
            total += y_batch.size(0)
            correct += predicted.eq(y_batch).sum().item()
    
    return total_loss / len(loader), 100. * correct / total

# 학습 실행
epochs = 15
best_acc = 0

print("개선된 MNIST MLP 학습")
print("="*60)

for epoch in range(epochs):
    train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, device)
    test_loss, test_acc = evaluate(model, test_loader, criterion, device)
    scheduler.step()
    
    if test_acc > best_acc:
        best_acc = test_acc
    
    print(f"Epoch {epoch+1:2d}/{epochs}: "
          f"Train Acc: {train_acc:.2f}% | Test Acc: {test_acc:.2f}% | LR: {scheduler.get_last_lr()[0]:.6f}")

print(f"\n최고 테스트 정확도: {best_acc:.2f}%")

In [ ]:
# 테스트
assert best_acc >= 98.0, f"98% 정확도 미달: {best_acc:.2f}%"
print(f"\n목표 달성! 테스트 정확도: {best_acc:.2f}% >= 98%")
print("모든 테스트 통과!")

### 풀이 설명

**접근 방법**: 모델 크기 증가, BatchNorm, 학습률 스케줄러를 조합합니다.

**핵심 개선사항**:
1. **은닉층 크기 증가**: 128,64 -> 512,256,128
2. **BatchNorm 추가**: 학습 안정화
3. **학습률 스케줄러**: StepLR로 점진적 감소
4. **에포크 증가**: 10 -> 15

**대안 솔루션**:
```python
# 데이터 증강 추가
transform = transforms.Compose([
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])
```

**흔한 실수**: 학습률 스케줄러 없이 높은 학습률 유지하면 후반에 진동합니다.

**실무 팁**: MLP로 MNIST 98%+는 가능하지만, CNN을 사용하면 99%+ 쉽게 달성 가능합니다.

---

### Q10. 종합: 이진 분류 MLP 파이프라인 

**문제**: sklearn의 breast cancer 데이터셋으로 이진 분류 MLP를 구축하세요.

요구사항:
1. 데이터 표준화 (StandardScaler)
2. MLP 모델 (은닉층 2개)
3. Adam 옵티마이저, BCE Loss
4. 학습 곡선 시각화
5. 테스트 정확도 출력

In [ ]:
# Q10 정답
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# 1. 데이터 로드 및 전처리
data = load_breast_cancer()
X, y = data.data, data.target

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# 표준화
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 텐서 변환
X_train_t = torch.tensor(X_train_scaled, dtype=torch.float32)
y_train_t = torch.tensor(y_train, dtype=torch.float32).reshape(-1, 1)
X_test_t = torch.tensor(X_test_scaled, dtype=torch.float32)
y_test_t = torch.tensor(y_test, dtype=torch.float32).reshape(-1, 1)

# DataLoader
train_dataset = TensorDataset(X_train_t, y_train_t)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

print(f"훈련 데이터: {X_train_t.shape}")
print(f"테스트 데이터: {X_test_t.shape}")
print(f"특성 수: {X_train_t.shape[1]}")
print(f"양성 비율 (훈련): {y_train.mean():.2%}")

In [ ]:
# 2. MLP 모델 정의
class BreastCancerMLP(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(32, 1),
            nn.Sigmoid()
        )
    
    def forward(self, x):
        return self.model(x)

# 모델, 손실 함수, 옵티마이저
model = BreastCancerMLP(input_dim=30)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

print("Breast Cancer MLP:")
print(model)

In [ ]:
# 3. 학습
epochs = 100
train_losses, train_accs = [], []
test_losses, test_accs = [], []

for epoch in range(epochs):
    # 훈련
    model.train()
    epoch_loss = 0
    for X_batch, y_batch in train_loader:
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
    
    # 평가
    model.eval()
    with torch.no_grad():
        # 훈련 정확도
        train_preds = (model(X_train_t) > 0.5).float()
        train_acc = (train_preds == y_train_t).float().mean().item()
        
        # 테스트 정확도
        test_outputs = model(X_test_t)
        test_loss = criterion(test_outputs, y_test_t).item()
        test_preds = (test_outputs > 0.5).float()
        test_acc = (test_preds == y_test_t).float().mean().item()
    
    train_losses.append(epoch_loss / len(train_loader))
    train_accs.append(train_acc * 100)
    test_losses.append(test_loss)
    test_accs.append(test_acc * 100)
    
    if (epoch + 1) % 20 == 0:
        print(f"Epoch {epoch+1:3d}: Train Acc: {train_acc*100:.2f}% | Test Acc: {test_acc*100:.2f}%")

print(f"\n최종 테스트 정확도: {test_accs[-1]:.2f}%")

In [ ]:
# 4. 학습 곡선 시각화
fig = make_subplots(rows=1, cols=2, subplot_titles=['손실 (Loss)', '정확도 (Accuracy)'])

fig.add_trace(go.Scatter(y=train_losses, mode='lines', name='Train Loss'), row=1, col=1)
fig.add_trace(go.Scatter(y=test_losses, mode='lines', name='Test Loss'), row=1, col=1)

fig.add_trace(go.Scatter(y=train_accs, mode='lines', name='Train Acc'), row=1, col=2)
fig.add_trace(go.Scatter(y=test_accs, mode='lines', name='Test Acc'), row=1, col=2)

fig.update_xaxes(title_text='Epoch')
fig.update_yaxes(title_text='Loss', row=1, col=1)
fig.update_yaxes(title_text='Accuracy (%)', row=1, col=2)
fig.update_layout(title='Breast Cancer MLP 학습 곡선', height=400, template='plotly_white')
fig.show()

In [ ]:
# 5. 최종 평가
from sklearn.metrics import classification_report

model.eval()
with torch.no_grad():
    test_preds = (model(X_test_t) > 0.5).numpy().astype(int).flatten()

print("분류 리포트:")
print(classification_report(y_test, test_preds, target_names=['악성', '양성']))

# 테스트
final_acc = (test_preds == y_test).mean() * 100
assert final_acc >= 90, f"정확도 90% 미달: {final_acc:.2f}%"
print(f"\n최종 정확도: {final_acc:.2f}%")
print("모든 테스트 통과!")

### 풀이 설명

**접근 방법**: End-to-End 파이프라인을 구축합니다.

**핵심 단계**:
1. **데이터 전처리**: StandardScaler로 표준화
2. **모델 설계**: 30 -> 64 -> 32 -> 1 MLP
3. **학습**: BCE Loss + Adam, 100 에포크
4. **시각화**: Plotly로 학습 곡선
5. **평가**: classification_report로 상세 분석

**대안 솔루션**:
```python
# 조기 종료 추가
best_loss = float('inf')
patience = 10
counter = 0

for epoch in range(epochs):
    ...
    if test_loss < best_loss:
        best_loss = test_loss
        counter = 0
        torch.save(model.state_dict(), 'best_model.pth')
    else:
        counter += 1
        if counter >= patience:
            print(f"Early stopping at epoch {epoch}")
            break
```

**흔한 실수**: 테스트 데이터에 fit_transform 적용하면 데이터 누수입니다. transform만 사용하세요.

**실무 팁**: 의료 데이터에서는 Recall(재현율)이 중요합니다. 암을 놓치지 않는 것이 오진보다 중요하기 때문입니다.

---

## 학습 정리

### Part 1: 기초 핵심 요약

| 개념 | 핵심 내용 | 실무 활용 |
|-----|----------|----------|
| 퍼셉트론 | y = f(W @ X + b) | 신경망의 기본 단위 |
| 활성화 함수 | 비선형성 추가 | ReLU (은닉), Sigmoid/Softmax (출력) |
| 손실 함수 | 예측 오차 측정 | MSE (회귀), CE (분류) |
| 경사 하강법 | w = w - lr * gradient | 손실 최소화 |
| XOR 문제 | 단일 퍼셉트론의 한계 | MLP로 해결 |

### Part 2: 심화 핵심 요약

| 개념 | 핵심 내용 | 언제 사용? |
|-----|----------|----------|
| 역전파 | 체인 룰로 그래디언트 계산 | 모든 신경망 학습 |
| MLP | 입력 -> 은닉층들 -> 출력 | 정형 데이터 분류/회귀 |
| SGD vs Adam | 기본 vs 적응적 학습률 | Adam이 기본, SGD는 일반화 |
| MNIST | 손글씨 분류 벤치마크 | 딥러닝 입문 |

### MLP 학습 핵심 패턴

```python
model = MLP(input_dim, hidden_dims, output_dim)
criterion = nn.CrossEntropyLoss()  # 분류
optimizer = optim.Adam(model.parameters(), lr=0.001)

for epoch in range(epochs):
    for X_batch, y_batch in dataloader:
        outputs = model(X_batch)          # 순전파
        loss = criterion(outputs, y_batch) # 손실 계산
        
        optimizer.zero_grad()   # 그래디언트 초기화
        loss.backward()         # 역전파
        optimizer.step()        # 파라미터 업데이트
```

### 실무 팁

1. **활성화 함수**: 은닉층에는 ReLU, 출력층은 문제에 맞게 (Sigmoid, Softmax, 없음)
2. **학습률**: Adam은 0.001, SGD는 0.01~0.1로 시작
3. **과적합 방지**: Dropout, 조기 종료, 데이터 증강
4. **배치 크기**: 32~128이 일반적, 메모리와 속도 트레이드오프
5. **디버깅**: 먼저 작은 데이터로 과적합 확인 (모델이 학습 가능한지)